In [1]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
from tensorflow import keras
import cv2
import pickle
from tensorflow.keras import backend as K
from PIL import Image
import matplotlib.patches as mpatches

In [2]:
PATH = '/kaggle/input/bridge-cracks-image/DeepPCB/PCBData'

In [3]:
normal = []
defect = []
defectlog = []
path_2 = [os.path.join(PATH, dir) for dir in os.listdir(PATH) if '.' not in dir]
for p in tqdm(path_2, total=len(path_2)):
    path_3 = os.path.join(p,sorted(os.listdir(p))[0])
    normal += [os.path.join(path_3,dir) for dir in os.listdir(path_3) if 'temp' in dir]
    defect += [os.path.join(path_3,dir) for dir in os.listdir(path_3) if 'test' in dir]
    path_4 = os.path.join(p,sorted(os.listdir(p))[1])
    defectlog += [os.path.join(path_4,dir) for dir in os.listdir(path_4)]

normal.sort()
defect.sort()
defectlog.sort()

In [4]:
img0 = []
img1 = []
for img_path in tqdm(normal,total=len(normal)):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (128, 128))
    img0.append(img)
for img_path in tqdm(defect,total=len(defect)):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (128, 128))
    img1.append(img)
img0 = np.array(img0)
img1 = np.array(img1)

In [5]:
box = []
for log in tqdm(defectlog, total=len(defectlog)):
    temp = pd.read_csv(log, sep=' ', names=['x1', 'y1', 'x2', 'y2', 'defect'], header=None).values.tolist()
    box.append(temp)
boxarr = []
for i in tqdm(range(len(box))):
    narr = np.zeros((640,640,1))
    for j in range(len(box[i])):
        x1,y1,x2,y2,d = box[i][j]      
        narr = cv2.rectangle(narr, (x1,y1),(x2,y2),d,-1)
    narr = cv2.resize(narr, (128,128))
    boxarr.append(narr)
boxarr = np.array(boxarr)

In [6]:
plt.imshow(np.reshape(boxarr[0],(128,128)),cmap='gnuplot')
plt.show()

In [7]:
plt.imshow(np.reshape(img1[0],(128,128)),cmap='gnuplot')
plt.show()

In [8]:
dataset = tf.data.Dataset.from_tensor_slices((img1,boxarr))

In [9]:
n=1500
dataset = dataset.shuffle(n)
train_dataset = dataset.take(int(n*0.8)).batch(100)
test_dataset = dataset.skip(int(n*0.8)).batch(100)

In [15]:
def conv2d_block(input_tensor, n_filters, kernel_size = 3, batchnorm = True):
    """Function to add 2 convolutional layers with the parameters passed to it"""
    # first layer
    x = tf.keras.layers.Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    if batchnorm:
        x = tf.keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation('relu')(x)
    
    # second layer
    x = tf.keras.layers.Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    if batchnorm:
        x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    
    return x
  
def get_unet(input_img, n_filters = 16, dropout = 0.1, batchnorm = True):
    
    x = keras.layers.experimental.preprocessing.Rescaling(1./255)(inputs)
    x = keras.layers.experimental.preprocessing.RandomFlip(mode='horizontal_and_vertical')(x)
    
    # Contracting Path
    c1 = conv2d_block(x, n_filters * 1, kernel_size = 3, batchnorm = batchnorm)
    p1 = keras.layers.MaxPooling2D((2, 2))(c1)
    p1 = keras.layers.Dropout(dropout)(p1)
    
    c2 = conv2d_block(p1, n_filters * 2, kernel_size = 3, batchnorm = batchnorm)
    p2 = keras.layers.MaxPooling2D((2, 2))(c2)
    p2 = keras.layers.Dropout(dropout)(p2)
    
    c3 = conv2d_block(p2, n_filters * 4, kernel_size = 3, batchnorm = batchnorm)
    p3 = keras.layers.MaxPooling2D((2, 2))(c3)
    p3 = keras.layers.Dropout(dropout)(p3)
    
    c4 = conv2d_block(p3, n_filters * 8, kernel_size = 3, batchnorm = batchnorm)
    p4 = keras.layers.MaxPooling2D((2, 2))(c4)
    p4 = keras.layers.Dropout(dropout)(p4)
    
    c5 = conv2d_block(p4, n_filters = n_filters * 16, kernel_size = 3, batchnorm = batchnorm)
    
    # Expansive Path
    u6 = tf.keras.layers.Conv2DTranspose(n_filters * 8, (3, 3), strides = (2, 2), padding = 'same')(c5)
    u6 = keras.layers.Concatenate()([u6, c4])
    u6 = keras.layers.Dropout(dropout)(u6)
    c6 = conv2d_block(u6, n_filters * 8, kernel_size = 3, batchnorm = batchnorm)
    
    u7 = tf.keras.layers.Conv2DTranspose(n_filters * 4, (3, 3), strides = (2, 2), padding = 'same')(c6)
    u7 = keras.layers.Concatenate()([u7, c3])
    u7 = keras.layers.Dropout(dropout)(u7)
    c7 = conv2d_block(u7, n_filters * 4, kernel_size = 3, batchnorm = batchnorm)
    
    u8 = tf.keras.layers.Conv2DTranspose(n_filters * 2, (3, 3), strides = (2, 2), padding = 'same')(c7)
    u8 = keras.layers.Concatenate()([u8, c2])
    u8 = keras.layers.Dropout(dropout)(u8)
    c8 = conv2d_block(u8, n_filters * 2, kernel_size = 3, batchnorm = batchnorm)
    
    u9 = tf.keras.layers.Conv2DTranspose(n_filters * 1, (3, 3), strides = (2, 2), padding = 'same')(c8)
    u9 = keras.layers.Concatenate()([u9, c1])
    u9 = keras.layers.Dropout(dropout)(u9)
    c9 = conv2d_block(u9, n_filters * 1, kernel_size = 3, batchnorm = batchnorm)
    
    outputs = keras.layers.Conv2D(filters=7,kernel_size=1)(c9)
    model = keras.Model(input_img, outputs)
    return model

In [16]:
inputs = keras.layers.Input((128,128,1))
model = get_unet(inputs)
model.summary()

In [17]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
)

In [18]:
history = model.fit(train_dataset,epochs=500,validation_data=test_dataset,verbose=1)

In [19]:
plt.plot(history.history['val_loss'])
plt.show()

In [20]:
out = model.predict(tf.reshape(img1[0],(-1,128,128,1)))
out2 = np.argmax(out,axis=3)
plt.imshow(out2[0],cmap='gnuplot')

In [26]:
img_arr_resized = cv2.resize(img1[0],(128,128))
img_arr_resized = cv2.cvtColor(img_arr_resized,cv2.COLOR_GRAY2RGB)

In [27]:
plt.imshow(img_arr_resized)

In [28]:
pred = model.predict(tf.reshape(img1[0],(-1,128,128,1)))
pred = np.argmax(pred,axis=3)[0]

In [29]:
new_pred = np.zeros((128,128,3),dtype=np.uint8)

In [30]:
new_pred[pred==0] = img_arr_resized[pred==0]
new_pred[pred==1] = [255,228,0] #open
new_pred[pred==2] = [1,0,255] #short
new_pred[pred==3] = [29,219,22] #mousebite
new_pred[pred==4] = [255,0,0] #spur
new_pred[pred==5] = [103,0,0] #copper
new_pred[pred==6] = [255,0,127] #pin-hole

In [31]:
blend_img1=Image.fromarray(img_arr_resized)
blend_img2=Image.fromarray(new_pred)
blend_img=Image.blend(blend_img1,blend_img2,0.5)

In [32]:
legend1 = mpatches.Patch(color='#FFE400', label='open')
legend2 = mpatches.Patch(color='#0100FF', label='short')
legend3 = mpatches.Patch(color='#1DDB16', label='mousebite')
legend4 = mpatches.Patch(color='#FF0000', label='spur')
legend5 = mpatches.Patch(color='#670000', label='copper')
legend6 = mpatches.Patch(color='#FF007F', label='pinhole')
plt.legend(loc='lower left', handles=[legend1,legend2,legend3,legend4,legend5,legend6],mode = "expand", ncol = 3,fontsize=10)
plt.xticks([])
plt.yticks([])
plt.imshow(np.array(blend_img))
plt.show()